# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import plotly.express as px
import numpy as np

cf.go_offline()

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [2]:
retail = pd.read_excel('../data/Online Retail.xlsx')
retail.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [3]:
data = retail.query('Country != "United Kingdom"')
data = data.query('InvoiceDate >= "2011-04-01" and InvoiceDate <= "2011-04-30"')

In [4]:
data = data.groupby('Country').agg({'Quantity':'sum','Revenue':'sum'})

In [5]:
cf.getThemes()

['ggplot', 'pearl', 'solar', 'space', 'white', 'polar', 'henanigans']

In [6]:
data.iplot(kind='bar',theme='pearl')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [7]:
data = retail.query('Country == "France" and InvoiceDate >= "2011-01-01" and InvoiceDate <= "2011-05-31"')

In [8]:
data = data.groupby('Country').agg({'Quantity':'sum','Revenue':'sum'})
data.iplot(kind='bar',sortbars=True,bargap=0.6,dimensions=(900,500))

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [9]:
data = retail[retail.Description.str.contains('PARTY BUNTING')]
data = data.groupby('Country').agg({'Quantity':'mean','UnitPrice':'mean'})

fig = px.scatter(data,x='Quantity',y='UnitPrice',color=data.index,hover_name=data.index)
fig.update_traces(marker_size = 12)

fig.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [10]:
countries = ['EIRE','Germany','France','Netherlands']

In [11]:
lst_data = [retail[retail.Country==country][['Country','InvoiceNo','Quantity']] for country in countries ]

In [12]:
lst_data = [df.groupby('InvoiceNo').agg({'Quantity':'sum'}).sort_index() for df in lst_data]

In [111]:
lst_data[0].head()

,Quantity
InvoiceNo,
536540,230
536541,12
536803,6
536890,1548
536975,827


In [14]:
px.histogram(lst_data[0],
             x=lst_data[0].index,
             y='Quantity',
             nbins=lst_data[0].shape[0])

In [15]:
from plotly.subplots import make_subplots    

colors =['#96FC65','#FF5D55','#5975FE','#E6C220']

# Inicializamos los subplots

fig_subplots = make_subplots(rows=2, cols=2,
                            subplot_titles=countries,
                            vertical_spacing=0.1)

# Recorremos la lista de df's

for index, df in enumerate(lst_data):
    fila = (index)//2 + 1       # Para asignar correctamente filas y columnas
    columna = (index) % 2 + 1
    fig = px.histogram(df,x=df.index,y='Quantity',nbins=df.shape[0],color_discrete_sequence=[colors[index]])
    for point in fig['data']:   # Tenemos que dibujar cada punto de fig en cada subplot
        fig_subplots.add_trace(point, row=fila , col=columna)

fig_subplots.update_layout(height=800,title={'text': 'Quantity per Invoice','x':0.5,'font': {'size': 40}}) # Ajustamos tamaño y ponemos título
fig_subplots.show() # mostramos

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [53]:
top_products = retail.StockCode.value_counts().index[:5].to_list()
data = retail[(retail.StockCode.isin(top_products))&(retail.Country.isin(countries))]
data = data.pivot_table(index='Country',
                        columns='StockCode',
                        values='Revenue',
                        aggfunc='sum')

In [59]:
px.bar(data,x=data.index, y=data.columns, barmode='group')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [88]:
data = retail[retail.Country=='United Kingdom']
data.InvoiceDate = data['InvoiceDate'].apply(lambda x:x.date())
data = data.groupby('InvoiceDate').agg({'Quantity':'sum'})
data.head()


,Quantity
InvoiceDate,
2010-12-01,21308
2010-12-02,30987
2010-12-03,7646
2010-12-05,13603
2010-12-06,15515
...,...
2011-12-05,35874
2011-12-06,25909
2011-12-07,29078


In [108]:
fig = px.line(data,x=data.index,y='Quantity')
fig.update_layout(xaxis=dict(rangeselector=dict(buttons=list([dict(count=1,label="1 mes",step="month",stepmode="backward"),
                                                              dict(count=6,label="6 meses",step="month",stepmode="backward"),
                                                              dict(count=1,label="Año en curso",step="year",stepmode="todate"),
                                                              dict(count=1,label="1 año",step="year",stepmode="backward"),
                                                              dict(label='Todo',step="all")])),
                             rangeslider=dict(visible=True),
                             type='date'),
                 title={'text': 'Quantity Vs Date','x':0.5,'font': {'size': 35}},
                 height=700)
fig.show()

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [118]:
data = retail.groupby('StockCode').agg({'CustomerID':'nunique','InvoiceNo':'count'})
data.head()

,CustomerID,InvoiceNo
StockCode,,
10002,40,49
10080,19,21
10120,25,30
10125,49,64
10133,101,124


In [126]:
fig = px.scatter(data,x='InvoiceNo',y='CustomerID',hover_name=data.index)
fig.show()

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.